# Tutorial 3: Probability propagation

#### `1.`  In tutorial no 3 we obtained the following Bayesian Network to model dyspnea.
![Dyspnea](images/dyspnea_network.png)
#### a) The network is intialised with prior probabilities allocated to $A$ and $S$ based on data about the local population. What $\lambda$ and $π$ messages will be sent during this initialisation?

- $\pi_T(A)$, $\pi_L(S)$, $\pi_B(S)$
- $T, L, B$ recalculate their posterior probabilities
- $\pi_X(T)$, $\pi_D(T)$, $\pi_X(L)$, $\pi_D(L)$, $\pi_D(B)$
- $X, B$ recalculate their posterior probabilities

#### b) A patient arrives, and the only fact that is known about him is that he is a smoker. Thus the node $S$ only is instantiated. What $\lambda$ and $\pi$ messages will be sent following the instantiation of $S$ ?

- $\pi_L(S)$, $\pi_B(S)$
- $L, B$ recalculate their posterior probabilities
- $\pi_X(L)$, $\pi_D(L)$, $\pi_D(B)$
- $X, D$ recalculate their posterior probabilities

#### c) The patient is given an XRay and it is found to be positive.  What $\lambda$ and $\pi$ messages will be sent following the instantiaion of $X$ ?

- $\lambda_X(T)$, $\lambda_X(L)$
- $T, L$ recalculate their posterior probabilities
- $\lambda_T(A)$, $\lambda_L(S)$ (ignored since S is already instantiated)
- $\pi_D(T)$, $\pi_D(L)$,
- $A, D$ recalculate their posterior probabilities

#### d) The patient is now examined further and found to suffer from dyspnea.  What would happen if we try to propagate the evidence under these circumstances?

Calculations do not terminate since a loop is created: $D - T - X - L - D$

#### `2.` As mentioned in the solution to tutorial 2, Neopolitan has a neat solution to the same causal network which is to introduce a new node standing for Lung Cancer or Tuberculosis.
![Dyspnea](images/dyspnea_network_metropolitan.png)
#### What would you expect the link matrix joining $T$ and $L$ with $LT$ to be?
Index 1 means the presence of given variable, 2 means absence.

\begin{equation*}
P(LT|T\&L) = \begin{pmatrix}
P(LT_1|T_1\&L_1) & P(LT_1|T_1\&L_2) & P(LT_1|T_2\&L_1) & P(LT_1|T_2\&L_2)\\
P(LT_2|T_1\&L_1) & P(LT_2|T_1\&L_2) & P(LT_2|T_2\&L_1) & P(LT_2|T_2\&L_2)
\end{pmatrix}
= 
\begin{pmatrix}
1 & 1 & 1 & 0\\ 
0 & 0 & 0 & 1
\end{pmatrix}
\end{equation*}

#### `3.` Given the net of question 2, and assuming the node $LT$ contains the following information:
$\lambda_X(LT) = (0.3,0.7) \\
\lambda_D(LT) = (1,1) \\
\pi_{LT}(T) = (0.4,0.6) \\
\pi_{LT}(L) = (0.5,0.5)$
#### What is the posterior probabilitiy distribution of $LT$? The node $T$ is suddenly instantiated to $T_2$ (tuberculosis is not present) what messages are sent from $LT$ (assuming that the conditional probability matrix is as given in question 2).

In [30]:
from numpy import *

P_LT_link_matrix = array([[1,1,1,0],[0,0,0,1]])

lambda_X_LT = array([0.3,0.7])
lambda_D_LT = array([1,1])
pi_LT_T = array([[0.4,0.6]])
pi_LT_L = array([[0.5,0.5]])

P_LT_joint = (transpose(pi_LT_T)).dot(pi_LT_L)
P_LT_joint_column_vector = transpose(array([hstack((P_LT_joint[0], P_LT_joint[1]))]))

pi_evidence_LT = P_LT_link_matrix.dot(P_LT_joint_column_vector)
posterior_LT = transpose(pi_evidence_LT)[0] * lambda_D_LT * lambda_X_LT
# Normalise
posterior_LT /= sum(posterior_LT)
print("Posterior of LT: ", posterior_LT)

Posterior of LT:  [ 0.5  0.5]


If $T$ is instantiated to $T_2$:

In [34]:
# Link matrix does not change
pi_LT_T = array([[0,1]])

P_LT_joint = (transpose(pi_LT_T)).dot(pi_LT_L)
P_LT_joint_column_vector = transpose(array([hstack((P_LT_joint[0], P_LT_joint[1]))]))

pi_evidence_LT = P_LT_link_matrix.dot(P_LT_joint_column_vector)

posterior_LT = transpose(pi_evidence_LT)[0] * lambda_D_LT * lambda_X_LT
# Normalise
posterior_LT /= sum(posterior_LT)
print("Posterior of LT: ", posterior_LT)

Posterior of LT:  [ 0.3  0.7]


In [50]:
pi_X_LT = posterior_LT/lambda_X_LT
pi_D_LT = posterior_LT/lambda_D_LT
lambda_LT = lambda_X_LT * lambda_D_LT

pi_LT_T = array([0,1])
lambda_LT_L1 = pi_LT_T[0] * (P_LT_link_matrix[0][0] * lambda_LT[0] + P_LT_link_matrix[1][0] * lambda_LT[1]) + pi_LT_T[1] * (P_LT_link_matrix[0][2] * lambda_LT[0] + P_LT_link_matrix[1][2] * lambda_LT[1])
lambda_LT_L2 = pi_LT_T[0] * (P_LT_link_matrix[0][1] * lambda_LT[0] + P_LT_link_matrix[1][1] * lambda_LT[1]) + pi_LT_T[1] * (P_LT_link_matrix[0][3] * lambda_LT[0] + P_LT_link_matrix[1][3] * lambda_LT[1])
lambda_LT_L = array([lambda_LT_L1, lambda_LT_L2])

print("Pi message from LT to X: ", pi_X_LT)
print("Pi message from LT to D: ", pi_D_LT) 
print("Lambda message from LT to L: ", lambda_LT_L )

Pi message from LT to X:  [ 1.  1.]
Pi message from LT to D:  [ 0.3  0.7]
Lambda message from LT to L:  [ 0.3  0.7]
